In [1]:
# Cell 1 - Tell python where to find the modules to import from

import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline if notebook runs from the repo root or from notebooks/
cwd = Path.cwd()
project_root = cwd if (cwd / "src").exists() else cwd.parent # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print("Project root: ", project_root)

Project root:  /Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


'--------------------------------  Test Load Data and Clean DataFrames  --------------------------------'

In [2]:
# Cell 2 — Imports, Load & Clean inputs (real paths)

from src import load_data
from src.clean_matrix import clean_matrix
from src.clean_relius_demo import clean_relius_demo
from src.age_taxcode_analysis import run_age_taxcode_analysis
from src.build_correction_file import build_correction_dataframe, write_correction_file

from src.config import RAW_DATA_DIR, USE_SAMPLE_DATA_DEFAULT


if USE_SAMPLE_DATA_DEFAULT:
    matrix_path = None
    relius_demo_path = None
else:
    matrix_path = RAW_DATA_DIR / "real_all_matrix_2025.xlsx"
    relius_demo_path = RAW_DATA_DIR / "real_demo_relius_2025.xlsx"


# Load  and clean Matrix raw data
matrix_raw = load_data.load_matrix_excel(path=matrix_path)
matrix_clean = clean_matrix(matrix_raw)

# Load and clean Relius demo raw data
relius_demo_raw = load_data.load_relius_demo_excel(path=relius_demo_path)
relius_demo_clean = clean_relius_demo(relius_demo_raw)


#.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
print("\n")
print(f"Matrix Cleaned DataFrame shape: {matrix_clean.shape}")
print(f"Relius Demo Cleaned DataFrame shape: {relius_demo_clean.shape}")



Matrix Cleaned DataFrame shape: (7, 21)
Relius Demo Cleaned DataFrame shape: (5, 11)


In [3]:
# Cell 3 — Review Dtype per columns and forst 10 ros of cleaned DataFrame

print(relius_demo_clean.info())
relius_demo_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3 to 1
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   plan_id            5 non-null      string 
 1   ssn                5 non-null      string 
 2   first_name         5 non-null      string 
 3   last_name          5 non-null      string 
 4   dob                3 non-null      object 
 5   term_date          4 non-null      object 
 6   ssn_valid          5 non-null      boolean
 7   amount_valid       0 non-null      boolean
 8   date_valid         4 non-null      boolean
 9   code_1099r_valid   0 non-null      boolean
 10  validation_issues  5 non-null      object 
dtypes: boolean(4), object(3), string(4)
memory usage: 360.0+ bytes
None


,plan_id,ssn,first_name,last_name,dob,term_date,ssn_valid,amount_valid,date_valid,code_1099r_valid,validation_issues
3,300004MBD,444556666,Noah,Garcia,NaT,2021-05-01,True,<NA>,True,<NA>,[]
0,300004PLAT,111223333,Ava,Nguyen,1970-05-10,2020-12-31,True,<NA>,True,<NA>,[]
2,300005R,333445555,Mia,Chen,1985-07-19,2023-06-30,True,<NA>,True,<NA>,[]
4,300005R,555667777,Zoe,Lopez,NaT,2022-01-15,True,<NA>,True,<NA>,[]
1,400001ABC,222334444,Liam,Patel,1962-11-03,NaT,True,<NA>,<NA>,<NA>,[]


'--------------------------------  Test Merging DataFrames and Analysis Engine --------------------------------'

In [4]:
# Cell 4 — Run Merging and Analysis Engine

age_matches = run_age_taxcode_analysis(matrix_clean, relius_demo_clean)

age_matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 6
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   plan_id                         5 non-null      string 
 1   ssn                             5 non-null      string 
 2   participant_name                5 non-null      object 
 3   state                           5 non-null      string 
 4   gross_amt                       5 non-null      float64
 5   fed_taxable_amt                 5 non-null      float64
 6   txn_date                        5 non-null      object 
 7   txn_method                      5 non-null      string 
 8   tax_code_1                      5 non-null      string 
 9   tax_code_2                      1 non-null      string 
 10  tax_form                        5 non-null      object 
 11  dist_type                       5 non-null      string 
 12  roth_initial_contribution_year  0 non-null   

In [5]:
# Cell 5 — Review tax codes in Series (1 or 2 digits)

age_matches[["tax_code_1"]].value_counts()

tax_code_1
7             3
1             1
ZZ            1
Name: count, dtype: int64

In [6]:
# Cell 6 — Review dtype of age at distribution Series and first at merged and analized DataFrame

print("age_at_distribution_year dtype: ", age_matches["age_at_distribution_year"].dtype)
age_matches.head(10)

age_at_distribution_year dtype:  Float64


,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code
0,300004PLAT,111223333,Ava Nguyen,CA,17179.62,17179.62,2024-01-17,ACH,7,G,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
1,400001ABC,222334444,Liam Patel,TX,8532.72,8532.72,2024-02-05,Wire,1,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
3,300004MBD,444556666,Noah Garcia,FL,4159.71,4159.71,2024-04-12,ACH,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
5,400001ABC,222334444,Liam Patel,TX,8991.28,8553.05,2099-01-01,ACH,7,<NA>,...,False,7,<NA>,<NA>,<NA>,match_no_action,True,<NA>,<NA>,<NA>
6,400001ABC,222334444,Liam Patel,TX,3000.00,9000.00,2024-02-20,Wire,ZZ,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7


In [7]:
# Cell 7 — Test no Roth plans in merged DataFrame

non_roth_test = age_matches[age_matches["plan_id"].str.startswith("300005") | age_matches["plan_id"].str.endswith("R")]
assert non_roth_test.empty, "WE HAVE ROTH DISBURSEMENTS!"

print("No Roth Disbursements in DataFrame!")

non_roth_test.head()

No Roth Disbursements in DataFrame!


,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code


In [8]:
# Cell 8 — Test rows with G tax code are excluded from analysis engine

rollover_test = age_matches[age_matches["tax_code_1"].eq("G")]
rollover_test.head()

,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code
20,IRA29PLAT,193361668,Robert Klemow,PA,218.92,0.0,2025-11-04,Check Distribution,G,<NA>,...,True,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
40,IRA29PLAT,161389368,Letitia Leitzel,FL,11.27,0.0,2025-10-06,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
73,IRA29PLAT,182601810,Thomas Richards,PA,200000.00,0.0,2025-09-02,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
93,IRA29PLAT,161389368,Letitia Leitzel,FL,15749.86,0.0,2025-07-31,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
155,IRA32PLAT,181466683,Martha Laux,PA,177377.67,0.0,2025-08-13,Check Reissue,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>


In [9]:
# Cell 9 — Review rows that match status is not 'match_no_action'

logic_test = age_matches[age_matches["match_status"] != "match_no_action"]

logic_test.head(15)

,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code
20,IRA29PLAT,193361668,Robert Klemow,PA,218.92,0.00,2025-11-04,Check Distribution,G,<NA>,...,True,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
40,IRA29PLAT,161389368,Letitia Leitzel,FL,11.27,0.00,2025-10-06,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
73,IRA29PLAT,182601810,Thomas Richards,PA,200000.00,0.00,2025-09-02,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
93,IRA29PLAT,161389368,Letitia Leitzel,FL,15749.86,0.00,2025-07-31,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
94,IRA29PLAT,168669091,Jared Fielder,VA,1013.52,1013.52,2025-07-30,ACH Distribution,4,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
95,IRA29PLAT,177649944,Brandon Fiedler,PA,1013.53,1013.53,2025-07-30,ACH Distribution,4,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
96,IRA29PLAT,185503634,Debra Garman,PA,2027.05,2027.05,2025-07-30,ACH Distribution,4,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
155,IRA32PLAT,181466683,Martha Laux,PA,177377.67,0.00,2025-08-13,Check Reissue,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
160,IRA32PLAT,181466683,Martha Laux,PA,177377.67,0.00,2025-07-28,Check Distribution,G,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,excluded_from_age_engine_rollover_or_inherited,False,<NA>,<NA>,<NA>
161,IRA32PLAT,202621585,Tonilee Rapp,PA,2117.65,2117.65,2025-07-03,ACH Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>


In [10]:
# Cell 10 — Review values and their counts in 'match_status'

age_matches["match_status"].value_counts()

match_status
match_no_action                                   5032
excluded_from_age_engine_rollover_or_inherited    1298
match_needs_correction                             290
age_rule_insufficient_data                          73
Name: count, dtype: int64

In [11]:
# Cell 11 — Review values and their counts in 'match_status' per unique SSN

filter_df = age_matches.drop_duplicates(subset="ssn")
print(filter_df["match_status"].value_counts())

filter_df[filter_df["match_status"] == "age_rule_insufficient_data"].head(15)

match_status
match_no_action                                   2115
excluded_from_age_engine_rollover_or_inherited    1009
match_needs_correction                              95
age_rule_insufficient_data                          30
Name: count, dtype: int64


,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code
161,IRA32PLAT,202621585,Tonilee Rapp,PA,2117.65,2117.65,2025-07-03,ACH Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
244,IRA109PLAT,131644147,Planned Parenthood Federation,NY,1000.00,1000.00,2025-08-11,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
304,IRA68PLAT,111111111,Food Bank Of Delaware,DE,600.00,600.00,2025-08-26,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
494,IRA147PLAT,251892186,Albert Gallatin Ed Found,PA,5000.00,5000.00,2025-09-04,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
735,300001MBD,232169885,Lycoming Valley Baptist Church,PA,100.00,100.00,2025-11-06,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
736,300001MBD,222856783,The Christian Gospel Fellowshi,PA,100.00,100.00,2025-11-06,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
737,300001MBD,205940449,I-TEC 23,PA,200.00,200.00,2025-11-06,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
738,300001MBD,232090782,Community Baptist Church of Mo,PA,500.00,500.00,2025-11-06,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
739,300001MBD,251458329,Eldred First Church of God,PA,740.00,740.00,2025-11-06,Check Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>
1107,300001MBD,160707297,Gerald Richardson,PA,6666.67,6666.67,2025-06-11,ACH Distribution,7,<NA>,...,False,<NA>,<NA>,<NA>,<NA>,age_rule_insufficient_data,False,<NA>,<NA>,<NA>


In [12]:
# Cell 12 — Review rows with 'match_needs_correction' status

age_taxcode = age_matches[age_matches["match_status"].eq("match_needs_correction")]

print(f" CORRECTION rows: {len(age_taxcode)}")
age_taxcode.head(15)

 CORRECTION rows: 290


,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code
240,IRA109PLAT,201407287,Paul Pfanders,PA,19480.52,0.00,2025-08-20,ACH Distribution,<NA>,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
743,300001MBD,195621676,Daun Boyle,PA,15000.00,15000.00,2025-11-05,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
885,300001MBD,206401373,Frances Vitovsky,PA,12639.65,0.00,2025-09-18,ACH Distribution,<NA>,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
938,300001MBD,195621676,Daun Boyle,PA,5000.00,5000.00,2025-09-03,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1000,300001MBD,167523906,Paul O'Hara III,PA,2750.00,2750.00,2025-08-01,ACH Distribution,3,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
1158,300003MBDII,221847676,Maria Ayala-McDonald,DE,653.60,653.60,2025-07-23,Wire Distribution,7,<NA>,...,False,1,<NA>,no_term_date_under_55_in_txn_year,UPDATE_1099,match_needs_correction,False,1,<NA>,1
1159,300003MBDII,221847676,Maria Ayala-McDonald,DE,11111.11,11111.11,2025-07-08,ACH Distribution,7,<NA>,...,False,1,<NA>,no_term_date_under_55_in_txn_year,UPDATE_1099,match_needs_correction,False,1,<NA>,1
1169,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.00,2025-11-06,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1180,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.00,2025-10-08,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1198,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.00,2025-09-03,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2


In [13]:
# Cell 13 — Review analysis and logic engine for rows were age at distribution >= 54 years old

age_taxcode[age_taxcode["age_at_distribution_year"].ge(54.0)].head(15)

,plan_id,ssn,participant_name,state,gross_amt,fed_taxable_amt,txn_date,txn_method,tax_code_1,tax_code_2,...,attained_55_in_term_year,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2,new_tax_code
240,IRA109PLAT,201407287,Paul Pfanders,PA,19480.52,0.0,2025-08-20,ACH Distribution,<NA>,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
743,300001MBD,195621676,Daun Boyle,PA,15000.00,15000.0,2025-11-05,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
885,300001MBD,206401373,Frances Vitovsky,PA,12639.65,0.0,2025-09-18,ACH Distribution,<NA>,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
938,300001MBD,195621676,Daun Boyle,PA,5000.00,5000.0,2025-09-03,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1000,300001MBD,167523906,Paul O'Hara III,PA,2750.00,2750.0,2025-08-01,ACH Distribution,3,<NA>,...,False,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099,match_needs_correction,False,7,<NA>,7
1169,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.0,2025-11-06,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1180,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.0,2025-10-08,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1198,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.0,2025-09-03,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1201,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.0,2025-08-08,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2
1209,IRA66PLAT,192644776,Constance Testa,PA,1625.00,1625.0,2025-08-05,ACH Distribution,1,<NA>,...,False,2,<NA>,no_term_date_55_plus_in_txn_year,UPDATE_1099,match_needs_correction,False,2,<NA>,2


'--------------------------------  Test Quick Export to Excel File --------------------------------'

In [ ]:
# Cell 14 — Use quick report export for manual DataFrame output to Excel for stakeholders

from src.export_utils import write_df_excel

path = write_df_excel(age_taxcode, filename_prefix="export_trad_distribs")

print(f"Export was successful!\nFile path: {path}")

'--------------------------------  Test Official Build/Export to Excel Correction File --------------------------------'

In [14]:
# Cell 15 — Use build_correction_file module to build the 'official' correction file

#Build Excel correction file (same build as inherited corrections engine)
age_correction_df = build_correction_dataframe(age_matches)

age_correction_df.shape

(290, 12)

Notes:
- Currently the age_correction DataFrame is finding all discrepancy between G and codes 1, 2 or G.
    - Age correction analysis is working, however code G is not based on age but in distribution type (G -> Rollovers).
    - We need to filter out distributions that are G (Traditional rollover) and H (Roth rollover) and exclude plans that are inherited since these are always code 4.
- Some tax codes are two digits (like '11'), I need to change the logi to extract 1 or two tax code digits.

In [ ]:
# Cell 16 — Use build_correction_file module to create/write/export the 'official' correction file

# 2) Write it to Excel with an auto-generated timestamped name
output_path = write_correction_file(age_correction_df)

# Run write_correction_file() function
output_path